In [1]:
#import needed things
from nltk.tokenize import sent_tokenize,word_tokenize,TweetTokenizer
from nltk.probability import FreqDist
import advertools as adv
from pathlib import Path
import string

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.naive_bayes import MultinomialNB,GaussianNB,BernoulliNB
from sklearn import linear_model

import codecs
import nltk
nltk.download('punkt')
import re
import emoji

from nltk.corpus import stopwords
from collections import Counter

from nltk.tokenize.treebank import TreebankWordDetokenizer as Detok
from nltk.tokenize.treebank import TreebankWordDetokenizer

F:\anaconda\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [75]:
df = pd.read_csv('data\combined_file.csv', encoding= 'unicode_escape')
df.head()

,Title,Category,Views,Keywords
0,Mastering Java Arrays: Comprehensive Guide,IT-Java,85K,"java, arrays, mastering, comprehensive guide, ..."
1,Java Array Manipulation Techniques,IT-Java,88K,"java, array manipulation, techniques, programming"
2,Efficient Java Array Operations,IT-Java,82K,"java, array operations, efficient, programming"
3,Java Array Sorting Algorithms,IT-Java,86K,"java, array sorting, algorithms, programming"
4,Dynamic Arrays in Java: Explained,IT-Java,84K,"java, dynamic arrays, explained, programming"


In [76]:
# identify the duplicate rows based on the Title column
duplicate_rows = df[df.duplicated(subset=['Title'], keep=False)]
# drop the duplicate rows
df = df.drop(duplicate_rows.index)
df['Title'] = df['Title'].str.lower()
df['Keywords'] = df['Keywords'].str.lower()
df.head()

,Title,Category,Views,Keywords
0,mastering java arrays: comprehensive guide,IT-Java,85K,"java, arrays, mastering, comprehensive guide, ..."
1,java array manipulation techniques,IT-Java,88K,"java, array manipulation, techniques, programming"
2,efficient java array operations,IT-Java,82K,"java, array operations, efficient, programming"
3,java array sorting algorithms,IT-Java,86K,"java, array sorting, algorithms, programming"
4,dynamic arrays in java: explained,IT-Java,84K,"java, dynamic arrays, explained, programming"


In [77]:
nltk.download('stopwords')
stopwords_set = set(stopwords.words('english'))
print(stopwords_set)

{'are', "that'll", 'didn', 'y', 'shouldn', 'just', 'and', 'into', 'whom', 'be', "aren't", 'because', 'other', 'mightn', 'both', 'further', 'was', 've', "weren't", "you'll", 'this', 'as', 'through', 'hers', 'has', 'during', 'she', 'not', 'will', 'ain', 'on', 'few', 'the', 'between', 'shan', 'up', 'their', 'wouldn', 'own', "mightn't", 'so', 'an', 'itself', 'before', 'with', 'him', 'does', 'about', 'in', "hadn't", "she's", "didn't", 'some', 'all', 'our', 'doesn', 'won', 'am', "couldn't", 'no', 'mustn', 'doing', 'is', 'but', "don't", 'now', 'isn', 'aren', 't', 'again', 'ourselves', 'these', 'under', "wouldn't", "hasn't", 'i', 'until', 'yours', 'its', 'only', 'them', 'or', 'it', 'ma', "shouldn't", 'after', 'herself', 'off', 'your', 'any', "should've", 'if', 'they', 'for', 'he', 're', 'that', 'you', 'by', "wasn't", 'me', 'yourself', 'themselves', 'from', 'himself', 'who', "you've", 'to', 'hadn', 'her', 'than', "doesn't", "shan't", 'did', 'over', 'we', 'very', 'those', 'wasn', 's', 'each', 'd

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [78]:
def remove_stopwords(text):
    tokens = nltk.word_tokenize(text)
    filtered_tokens = [token for token in tokens if token not in stopwords_set]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

df['Title'] = df['Title'].apply(remove_stopwords)

In [79]:
def remove_numbers(title):
    return re.sub(r'\d', '', title)

# Apply the function to the 'text' column
df['Title'] = df['Title'].apply(remove_numbers)

In [80]:
# Remove leading spaces before text in the specified column
df['Title'] = df['Title'].str.lstrip()
df['Keywords'] = df['Keywords'].str.lstrip()

df = df.replace(":", "", regex=True)
df = df.replace("s'", "", regex=True)

In [81]:
df = df.replace("-", "", regex=True)
df = df.replace(",","", regex=True)

# Remove leading spaces before text in the specified column
df['Title'] = df['Title'].str.lstrip()

In [82]:
# Save the cleaned DataFrame to a new CSV file
df.to_csv('data\cleaned_dataset.csv', index=False)
df

,Title,Category,Views,Keywords
0,mastering java arrays comprehensive guide,ITJava,85K,java arrays mastering comprehensive guide prog...
1,java array manipulation techniques,ITJava,88K,java array manipulation techniques programming
2,efficient java array operations,ITJava,82K,java array operations efficient programming
3,java array sorting algorithms,ITJava,86K,java array sorting algorithms programming
4,dynamic arrays java explained,ITJava,84K,java dynamic arrays explained programming
...,...,...,...,...
1165,sorting algorithms java predictive analytics,ITJava,98K,java sorting algorithms predictive analytics
1166,java sorting algorithms internet things ( iot...,ITJava,96K,java sorting algorithms internet of things iot...
1167,sorting algorithms java natural language proc...,ITJava,93K,java sorting algorithms natural language proce...
1168,java sorting algorithms quantum computing,ITJava,97K,java sorting algorithms quantum computing
